In [ ]:
import pandas as pd
import joblib
import ta
import ccxt
from datetime import timedelta

CSV_PATH = r"C:\Users\krish\OneDrive\Desktop\BITCOIN FINAL FOLDER FOR PREDICTION!\testing!.csv"
MODEL_PATH = r"C:\Users\krish\OneDrive\Desktop\BITCOIN FINAL FOLDER FOR PREDICTION!\btc_ml_oos_model_by_krish.pkl"

SYMBOL = "BTC/USDT"
TIMEFRAME = "1d"


def run_prediction():
    exchange = ccxt.binance({"enableRateLimit": True})

    df_existing = pd.read_csv(CSV_PATH)
    df_existing["Date"] = pd.to_datetime(df_existing["Date"], dayfirst=True)

    last_date = df_existing["Date"].max()
    since = int((last_date + timedelta(days=1)).timestamp() * 1000)

    try:
        ohlcv = exchange.fetch_ohlcv(SYMBOL, TIMEFRAME, since=since, limit=1000)
    except:
        ohlcv = []

    if ohlcv:
        df_new = pd.DataFrame(
            ohlcv,
            columns=["Timestamp", "Open", "High", "Low", "Close", "Volume"]
        )
        df_new["Date"] = pd.to_datetime(df_new["Timestamp"], unit="ms")
        df_new["Date"] = df_new["Date"].dt.strftime("%d-%m-%Y")
        df_new = df_new[["Date", "Open", "High", "Low", "Close", "Volume"]]

        df_existing["Date"] = df_existing["Date"].dt.strftime("%d-%m-%Y")
        df_existing = pd.concat([df_existing, df_new], ignore_index=True)
        df_existing.to_csv(CSV_PATH, index=False)

    btc = pd.read_csv(CSV_PATH)
    btc["Date"] = pd.to_datetime(btc["Date"], dayfirst=True)
    btc.set_index("Date", inplace=True)
    btc.sort_index(inplace=True)

    for col in ["Open", "High", "Low", "Close", "Volume"]:
        btc[col] = pd.to_numeric(btc[col], errors="coerce")

    btc.dropna(inplace=True)

    # FEATURES
    btc["ret_1d"] = btc["Close"].pct_change()
    btc["ret_5d"] = btc["Close"].pct_change(5)
    btc["ret_20d"] = btc["Close"].pct_change(20)

    btc["vol_10d"] = btc["ret_1d"].rolling(10).std()
    btc["vol_30d"] = btc["ret_1d"].rolling(30).std()

    btc["rsi"] = ta.momentum.RSIIndicator(btc["Close"], 14).rsi()

    btc["ema_fast"] = ta.trend.EMAIndicator(btc["Close"], 20).ema_indicator()
    btc["ema_slow"] = ta.trend.EMAIndicator(btc["Close"], 50).ema_indicator()

    btc["trend"] = (btc["ema_fast"] - btc["ema_slow"]) / btc["ema_slow"]
    btc["price_vs_ema"] = (btc["Close"] - btc["ema_slow"]) / btc["ema_slow"]
    btc["bull_regime"] = (btc["Close"] > btc["ema_slow"]).astype(int)

    btc.dropna(inplace=True)

    FEATURES = [
        "ret_5d", "ret_20d", "vol_10d", "vol_30d",
        "rsi", "trend", "price_vs_ema", "bull_regime"
    ]

    model = joblib.load(MODEL_PATH)

    latest = btc.iloc[-1:][FEATURES]
    prob = model.predict_proba(latest)[0, 1]

    regime = "Bullish" if latest["bull_regime"].iloc[0] == 1 else "Bearish"

    if prob >= 0.70:
        confidence = "High"
    elif prob >= 0.55:
        confidence = "Moderate"
    else:
        confidence = "Low"

    if prob >= 0.65 and regime == "Bullish":
        action = "HOLD / ENTER LONG"
        color = "green"
    else:
        action = "STAY OUT"
        color = "red"

    today = latest.index[0].date()
    start = today + timedelta(days=1)
    end = start + timedelta(days=4)

    return {
        "date": str(today),
        "regime": regime,
        "probability": round(prob * 100, 2),
        "confidence": confidence,
        "action": action,
        "color": color,
        "window": f"{start} → {end}"
    }
